# LArPix-v2 10x10 Tile Data Analysis

In [1]:
import os.path
import h5py
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interact_manual
from glob import glob
from IPython.display import display
import pandas as pd
from datetime import datetime
from tqdm import tqdm_notebook

## Raw data

In [2]:
directory = "/project/projectdirs/dune/data/larpix/raw_data/raw_20_10_12/*/"
filepaths = glob(directory+"*.h5")
dirs = list(map(os.path.dirname, filepaths))
filenames = list(map(os.path.basename, filepaths))
file_dict = dict(zip(filenames,dirs))

In [3]:
filename = filenames[0]
f = h5py.File(file_dict[filename]+"/"+filename,'r')
df_packets = pd.DataFrame(np.array(f['packets']))
df_packets["unique_channel"] = df_packets["channel_id"].astype("int64") + \
                               64*(df_packets["chip_id"].astype("int64") + \
                                   256*(df_packets["io_channel"].astype("int64") + \
                                        256*df_packets["io_group"].astype("int64")))

unique_channels = list(df_packets.groupby("unique_channel").indices.keys())

### Channel ADC values

In [4]:
@interact_manual
def test(filenames=widgets.SelectMultiple(options=filenames,
                                          value=[filenames[0]],
                                          description="File", 
                                          layout=widgets.Layout(width='30%'))):
    if filenames:
        
        plt.figure("Channel ADC values",figsize=(10,8))
        gs = gridspec.GridSpec(2,1)
        gs.update(hspace=0.05)
        ax0 = plt.subplot(gs[0])
        ax1 = plt.subplot(gs[1])
        
        for filename in tqdm_notebook(filenames):
            f = h5py.File(file_dict[filename]+"/"+filename,'r')
            ts = f['_header'].attrs['created']
            date_label = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            df_packets = pd.DataFrame(np.array(f['packets']))
            df_packets["unique_channel"] = df_packets["channel_id"].astype("int64") + \
                                           64*(df_packets["chip_id"].astype("int64") + \
                                               256*(df_packets["io_channel"].astype("int64") + \
                                                    256*df_packets["io_group"].astype("int64")))
            
            unique_channels = list(df_packets.groupby("unique_channel").indices.keys())
            ax0.scatter(unique_channels, 
                          df_packets.groupby("unique_channel")["dataword"].mean(),
                          alpha=0.5,
                          label=date_label)
            ax0.set_xticklabels([])
            ax1.scatter(unique_channels, 
                          df_packets.groupby("unique_channel")["dataword"].std(),
                          alpha=0.5,
                          label=date_label)
            
        ax1.set_xlabel("Unique channel ID")
        ax0.set_ylabel("Mean ADC value")
        ax1.set_ylabel("Std ADC value")
        ax0.legend(ncol=2,bbox_to_anchor=(1.05, 1))

interactive(children=(SelectMultiple(description='File', index=(0,), layout=Layout(width='30%'), options=('dat…

### Channel ADC

In [5]:
@interact
def test(filename=widgets.Select(options=filenames, description="File", layout=widgets.Layout(width='30%')),
         channel_id=widgets.BoundedIntText(value=0, min=0, max=64,description='Channel ID:'),
         chip_id=widgets.BoundedIntText(value=0, min=11, max=200,description='Chip ID:'),
         io_channel=widgets.BoundedIntText(value=0, min=1, max=4,description='IO channel:'),
         io_group=widgets.BoundedIntText(value=1, min=1, max=1,description='IO group:')):
    try:
        f = h5py.File(file_dict[filename]+"/"+filename,'r')
        df_packets = pd.DataFrame(np.array(f['packets']))
        
        plt.close('all')
        adc = df_packets.query("(channel_id == %i) & (chip_id == %i) & (io_channel == %i) & (io_group == %i)" 
                               % (channel_id, chip_id, io_channel, io_group))["dataword"]
        unique_channel = channel_id + 64*(chip_id + 256*(io_channel + 256*io_group))
        if adc.any():
            fig, ax = plt.subplots(1,1)
            hist, bins, patches = ax.hist(adc)
            ax.set_xlabel("ADC value")
        else:
            print("Can't find unique channel", unique_channel)
    except OSError:
        print("Invalid HDF5 file", filename)
    except TypeError:
        print("No available files")

interactive(children=(Select(description='File', layout=Layout(width='30%'), options=('datalog_2020_10_12_22_5…

## Processed data

In [6]:
directory_proc = "/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_12/"
filepaths_proc = glob(directory_proc+"*.h5")
filenames_proc = list(map(os.path.basename, filepaths_proc))

@interact
def test(filename=widgets.Dropdown(options=filenames_proc, description="File"),
         event_idx=widgets.IntText(0, description="Event ID")):
    try:
        f = h5py.File(directory_proc+filename,'r')

        hit_ref = f['events'][event_idx]['hit_ref']
        hit_x = f['hits'][hit_ref]['px'] 
        hit_y = f['hits'][hit_ref]['py'] 
        hit_rel_ts = f['hits'][hit_ref]['ts'] - f['events'][event_idx]['ts_start']
        hit_q = f['hits'][hit_ref]['q'] * 0.250

        fig = plt.figure(figsize=(8,5),num=' ')
        fig.clf()

        ax1 = fig.add_subplot(2, 3, 1)
        ax2 = fig.add_subplot(2, 3, 2)
        ax3 = fig.add_subplot(2, 3, 3, projection='3d')
        ax4 = fig.add_subplot(2, 3, 4)
        ax5 = fig.add_subplot(2, 3, 5)
        ax6 = fig.add_subplot(2, 3, 6)

        ax1.hist(hit_rel_ts,histtype='step')
        ax1.set_xlabel("$\Delta t$ [clock ticks]")

        ax2.hist2d(hit_x,hit_y)
        ax2.set_xlabel("$x$ [mm]")
        ax2.set_ylabel("$y$ [mm]")

        ax3.scatter(hit_x, hit_y, hit_rel_ts, 
                    alpha=1,
                    c=hit_q)
        ax3.set_xlabel("$x$ [mm]")
        ax3.set_ylabel("$y$ [mm]")
        ax3.set_zlabel("$\Delta t$ [clock ticks]")

        ax4.hist(hit_x,histtype='step')
        ax4.set_xlabel("$x$ [mm]")

        ax5.hist(hit_y,histtype='step')
        ax5.set_xlabel("$y$ [mm]")

        ax6.hist(hit_q,histtype='step')
        ax6.set_xlabel("Charge [# electrons]")

        fig.tight_layout()
    except ValueError:
        print("Invalid Event ID", event_idx)
    except OSError:
        print("Invalid HDF5 file", filename)
    except TypeError:
        print("No available files")

interactive(children=(Dropdown(description='File', options=('datalog_2020_10_12_18_09_31_PDT_evd.h5', 'datalog…

In [7]:
@interact_manual
def test(filenames=widgets.SelectMultiple(options=filenames_proc,
                                          value=[filenames_proc[0]],
                                          description="File", 
                                          layout=widgets.Layout(width='35%'))):
    if filenames:
        
        fig, ax = plt.subplots(1,1)
        for filename in tqdm_notebook(filenames):
            f = h5py.File(directory_proc+"/"+filename,'r')
#             date_label = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            ax.hist(f['hits']['q'],bins=100,histtype='step',range=(-200,400),label=filename)
            
            
        ax.set_xlabel("Charge")
        ax.set_yscale("log")
        ax.legend(ncol=2,bbox_to_anchor=(1.05, 1))
#         ax0.set_ylabel("Mean ADC value")
#         ax1.set_ylabel("Std ADC value")
#         ax0.legend(ncol=2,bbox_to_anchor=(1.05, 1))

interactive(children=(SelectMultiple(description='File', index=(0,), layout=Layout(width='35%'), options=('dat…